## Task 3: Data extraction and entity resolution using Python

#### Task 3(a)

In [5]:
import sqlite3
import pandas

path = r"bank.db" #path to db

#try to establish connection
try:
    conn = sqlite3.connect(path)
except Exception:
    raise ConnectionError

#create cursor for excecution on db
cur = conn.cursor()

#queries from queries.sql in python string format
queries = [
    """SELECT *
    FROM employee
    LEFT OUTER JOIN branch
    ON employee.branch_id = branch.branch_id;""",

    """SELECT branch_id, AVG(salary)
    FROM employee
    GROUP BY branch_id;""",

    """SELECT branch_id, avg_salary
    FROM (
            SELECT branch_id, AVG(salary) AS avg_salary
            FROM employee
            GROUP BY branch_id
        )
    WHERE avg_salary > 35000;"""
]

#loop over queries and fetch results
for query in queries:
    try:
        #result
        res = cur.execute(query)   #excecute query
        res = res.fetchall()       #retrieve all results
        results = [i for i in res] #put it into python list
        print(results)
    except Exception:
        print("error")
        continue


#close connection
conn.close()

[(1, 'Stanley', 'Hudson', 'Sales Representative', 45000, '2005-04-01', 1, 1, 'Dunder Mifflin Banking Company, Inc', 'Slough Avenue', 20, 'Scranton', '717 555 0177', '9-17'), (2, 'Kevin', 'Malone', 'Accountant', 42000, '2007-06-15', 1, 1, 'Dunder Mifflin Banking Company, Inc', 'Slough Avenue', 20, 'Scranton', '717 555 0177', '9-17'), (3, 'Greg', 'Lestrade', 'Detective Inspector', 50000, '2010-02-20', 2, 2, 'Baking and Banking', 'Bakerstreet', 221, 'London', '010 737 1373', '9-17'), (4, 'Molly', 'Hooper', 'Pathologist', 48000, '2011-09-25', 4, None, None, None, None, None, None, None), (5, 'Pearl', 'Krabs', 'Intern', 12000, '2022-07-01', 3, 3, 'Krusty Krab Bank', 'Anchor Way', 1, 'Bikini Bottom', '555 0199', '9-17'), (6, 'Sheldon', 'Plankton', 'Competitor Spy', 30000, '2023-01-10', 3, 3, 'Krusty Krab Bank', 'Anchor Way', 1, 'Bikini Bottom', '555 0199', '9-17')]
[(1, 43500.0), (2, 50000.0), (3, 21000.0), (4, 48000.0)]
[(1, 43500.0), (2, 50000.0), (4, 48000.0)]


### Task 3(b)

In [6]:
import sqlite3
import pandas as pd

# Path to the database
path = r"bank.db"  # Adjust the path if necessary

# Try to establish a connection to the database
try:
    conn = sqlite3.connect(path)
except Exception:
    raise ConnectionError("Failed to connect to the database.")

# SQL query to select all data from the customer table
query = """SELECT * FROM customer"""

# Execute the query and load the results into a Pandas DataFrame
try:
    df_customers = pd.read_sql_query(query, conn)  # Read the table into a DataFrame
    print(df_customers.head())  # Display the first few rows as a preview
except Exception as e:
    print(f"Error while executing the query: {e}")

# Close the connection
conn.close()


   customer_id first_name second_name         street  house_number      city  \
0            1    Michael       Scott  Slough Avenue          1725  Scranton   
1            2        Jim     Halpert   Beech Street           236  Scranton   
2            3        Pam      Beesly    Main Street           411  Scranton   
3            4     Dwight     Schrute  Walnut Street           700  Scranton   
4            5     Angela      Martin   Maple Street           300  Scranton   

  country  phone_number date_of_birth  
0     USA  717-555-0101    1979-03-15  
1     USA  717-555-0102    1981-10-14  
2     USA  717-555-0103    1985-06-27  
3     USA  717-555-0104    1980-02-19  
4     USA  717-555-0105    1978-01-25  


### Task 3(c)

In [7]:
# Jaro similarity function
def jaro(s, t):
    s_len = len(s)
    t_len = len(t)
    
    if s_len == 0 and t_len == 0:
        return 1
    
    match_distance = (max(s_len, t_len) // 2) - 1
    
    s_matches = [False] * s_len
    t_matches = [False] * t_len
    
    matches = 0
    transpositions = 0
    
    for i in range(s_len):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, t_len)
        
        for j in range(start, end):
            if t_matches[j]:
                continue
            if s[i] != t[j]:
                continue
            s_matches[i] = True
            t_matches[j] = True
            matches += 1
            break
    
    if matches == 0:
        return 0
    
    k = 0
    for i in range(s_len):
        if not s_matches[i]:
            continue
        while not t_matches[k]:
            k += 1
        if s[i] != t[k]:
            transpositions += 1
        k += 1
    
    return ((matches / s_len) +
            (matches / t_len) +
            ((matches - transpositions / 2) / matches)) / 3

# Function to compare customers and report those with similarity > 0.7
def compare_customers(df):
    num_customers = len(df)
    for i in range(num_customers):
        for j in range(i + 1, num_customers):
            # Combine the columns (first_name, second_name, city, phone_number, and date_of_birth) into strings
            customer1 = ' '.join(df.iloc[i].astype(str).values)
            customer2 = ' '.join(df.iloc[j].astype(str).values)
            
            # Calculate similarity between the two customers using Jaro similarity
            similarity = jaro(customer1, customer2)
            
            # Report customers with similarity > 0.7
            if similarity > 0.7:
                print(f"Customer {i+1} and Customer {j+1} have a similarity score of {similarity:.2f}")

# Apply the comparison function to df_customers
compare_customers(df_customers)


Customer 1 and Customer 4 have a similarity score of 0.73
Customer 1 and Customer 5 have a similarity score of 0.73
Customer 2 and Customer 3 have a similarity score of 0.76
Customer 2 and Customer 4 have a similarity score of 0.72
Customer 2 and Customer 5 have a similarity score of 0.73
Customer 2 and Customer 6 have a similarity score of 0.70
Customer 3 and Customer 5 have a similarity score of 0.78
Customer 4 and Customer 5 have a similarity score of 0.72
Customer 6 and Customer 7 have a similarity score of 0.72
Customer 6 and Customer 8 have a similarity score of 0.75
Customer 11 and Customer 12 have a similarity score of 0.76
Customer 11 and Customer 13 have a similarity score of 0.75
Customer 11 and Customer 14 have a similarity score of 0.76
Customer 11 and Customer 15 have a similarity score of 0.72
Customer 12 and Customer 13 have a similarity score of 0.76
Customer 12 and Customer 14 have a similarity score of 0.72
Customer 12 and Customer 15 have a similarity score of 0.72
